![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

In [17]:
# Import necessary libraries
import pandas as pd
import numpy as np

# Load the dataset
ds_jobs = pd.read_csv("customer_train.csv")

# View the dataset
ds_jobs.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [18]:
# Create a copy of ds_jobs for transforming
ds_jobs_transformed = ds_jobs.copy()
ds_jobs_transformed.head()
# Start coding here. Use as many cells as you like!

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


### Functions

In [19]:
def remove_list_nans(lst:list):
    return ', '.join(['' if val is np.nan else str(val) for val in lst])

def get_unique_vals(df:pd.DataFrame, category_threshold:int=10):
    return [remove_list_nans(list(df[col].unique())) if df[col].nunique() <= category_threshold else [] for col in df.columns]

def dtype_details(df:pd.DataFrame, category_threshold:int=10, specific:str=None) -> pd.DataFrame:
    max_col_len = len( max( list(df.columns), key=len) )+1
    max_num_len = len( str( max( list(df.nunique()) ) ) )
    df_return = pd.DataFrame(df.dtypes, columns=['dtype'])
    df_return['unique'] = list(df.nunique())
    unique_vals = get_unique_vals(df, category_threshold)
    df_return['unique_values'] = unique_vals
    if specific:
        return df_return[df_return['dtype']==specific]
    else:
        return df_return

def change_type(df:pd.DataFrame, cols:list=None, from_type:str=None, to_type:str='str') -> pd.core.dtypes:
    df_details = dtype_details(ds_jobs_transformed, 10)
    if not cols:
        cols = df_details[df_details['dtype']==from_type].index.values
    ret= cols
    for col in cols:
        df[col] = ds_jobs_transformed[col].astype(to_type)
    return df[ret].dtypes

def change_to_category(df:pd.DataFrame, cols: list, map_val:dict=None, category_list:list=None, order:bool=None, nan_replacer:str='None') -> None:
    for col in cols:
        df[col] = df[col].astype('category')
        if map_val:
            df[col] = df[col].cat.rename_categories(map_val)
        if category_list:
            df[col] = df[col].cat.set_categories(new_categories = category_list)
            df.loc[df[col].isna(), col] = nan_replacer
        if order:
            df[col] = df[col].cat.reorder_categories(new_categories = category_list, ordered=order)
            df.loc[df[col].isna(), col] = nan_replacer

### Columns containing categories with only two factors must be stored as Booleans (bool)

In [20]:
dtype_details(ds_jobs_transformed)[dtype_details(ds_jobs_transformed)['unique']<=2]

,dtype,unique,unique_values
relevant_experience,object,2,"Has relevant experience, No relevant experience"
job_change,float64,2,"1.0, 0.0"


In [21]:
map_value = {
    'Has relevant experience':True
    , 'No relevant experience':False
}
change_to_category(ds_jobs_transformed, ['relevant_experience'], map_value)
map_value = {
    1.0:True
    , 0.0:False
}
change_to_category(ds_jobs_transformed, ['job_change'], map_value)

In [22]:
ds_jobs_transformed[['relevant_experience', 'job_change']].dtypes

relevant_experience    category
job_change             category
dtype: object

In [23]:
col_list = ['relevant_experience', 'job_change']
change_type(ds_jobs_transformed, cols=col_list, to_type='bool')
ds_jobs_transformed[col_list].dtypes

relevant_experience    bool
job_change             bool
dtype: object

### Columns containing integers only must be stored as 32-bit integers (int32).

In [24]:
dtype_details(ds_jobs_transformed, specific='int64')

,dtype,unique,unique_values
student_id,int64,19158,[]
training_hours,int64,241,[]


In [25]:
change_type(ds_jobs_transformed, from_type='int64', to_type='int32')

student_id        int32
training_hours    int32
dtype: object

### Columns containing floats must be stored as 16-bit floats (float16)

In [26]:
dtype_details(ds_jobs_transformed, specific='float64')

,dtype,unique,unique_values
city_development_index,float64,93,[]


In [27]:
change_type(ds_jobs_transformed, from_type='float64', to_type='float16')

city_development_index    float16
dtype: object

### Columns containing nominal categorical data must be stored as the category data type.

In [28]:
dtype_details(ds_jobs_transformed)[(dtype_details(ds_jobs_transformed)['unique']<=10)&(dtype_details(ds_jobs_transformed)['dtype']!='category')]

,dtype,unique,unique_values
gender,object,3,"Male, , Female, Other"
relevant_experience,bool,2,"True, False"
enrolled_university,object,3,"no_enrollment, Full time course, , Part time c..."
education_level,object,5,"Graduate, Masters, High School, , Phd, Primary..."
major_discipline,object,6,"STEM, Business Degree, , Arts, Humanities, No ..."
company_size,object,8,", 50-99, <10, 10000+, 5000-9999, 1000-4999, 10..."
company_type,object,6,", Pvt Ltd, Funded Startup, Early Stage Startup..."
last_new_job,object,6,"1, >4, never, 4, 3, 2,"
job_change,bool,2,"True, False"


In [29]:
change_to_category(
    ds_jobs_transformed
    , ['gender']
    , category_list=['Male', 'Female', 'Other']
    , order=False
    , nan_replacer='Other'
)
change_to_category(
    ds_jobs_transformed
    , ['major_discipline']
    , category_list=['None', 'STEM', 'Business Degree''Arts', 'Humanities', 'No Major', 'Other']
    , order=False
    , nan_replacer='None'
)
change_to_category(
    ds_jobs_transformed
    , ['company_type']
    , category_list=['None', 'Pvt Ltd', 'Funded Startup', 'Early Stage Startup', 'Other','Public Sector', 'NGO']
    , order=False
    , nan_replacer='None'
)

In [ ]:
change_type(ds_jobs_transformed, cols=['city'], to_type='category')

In [30]:
ds_jobs_transformed[['gender', 'enrolled_university', 'major_discipline', 'company_type']].dtypes

gender                 category
enrolled_university    category
major_discipline       category
company_type           category
dtype: object

### Columns containing ordinal categorical data must be stored as ordered categories, and not mapped to numerical values, with an order that reflects the natural order of the column.

In [31]:
dtype_details(ds_jobs_transformed, category_threshold=25)[(dtype_details(ds_jobs_transformed)['unique']<=25)&(dtype_details(ds_jobs_transformed)['dtype']!='category')]

,dtype,unique,unique_values
relevant_experience,bool,2,"True, False"
education_level,object,5,"Graduate, Masters, High School, , Phd, Primary..."
experience,object,22,">20, 15, 5, <1, 11, 13, 7, 17, 2, 16, 1, 4, 10..."
company_size,object,8,", 50-99, <10, 10000+, 5000-9999, 1000-4999, 10..."
last_new_job,object,6,"1, >4, never, 4, 3, 2,"
job_change,bool,2,"True, False"


In [32]:
change_to_category(
    ds_jobs_transformed
    , ['education_level']
    , category_list=['None', 'Primary School', 'High School', 'Graduate', 'Masters', 'Phd']
    , order=True
    , nan_replacer='None'
)
change_to_category(
    ds_jobs_transformed
    , ['enrolled_university']
    , category_list=['no_enrollment', 'Part time course', 'Full time course']
    , order=True
    , nan_replacer='no_enrollment'
)
change_to_category(
    ds_jobs_transformed
    , ['experience']
    , category_list=['None', '<1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '>20']
    , order=True
    , nan_replacer='None'
)
change_to_category(
    ds_jobs_transformed
    , ['company_size']
    , category_list=['None', '<10', '10-49', '50-99', '100-499', '500-999', '1000-4999', '5000-9999', '10000+']
    , order=True
    , nan_replacer='None'
)
change_to_category(
    ds_jobs_transformed
    , ['last_new_job']
    , category_list=['never', '1', '2', '3', '4','>4']
    , order=True
    , nan_replacer='never'
)

In [33]:
ds_jobs_transformed[['education_level', 'experience', 'company_size', 'last_new_job']].dtypes

education_level    category
experience         category
company_size       category
last_new_job       category
dtype: object

In [34]:
ds_jobs_transformed['education_level'].unique()

['Graduate', 'Masters', 'High School', 'None', 'Phd', 'Primary School']
Categories (6, object): ['None' < 'Primary School' < 'High School' < 'Graduate' < 'Masters' < 'Phd']

### The DataFrame should be filtered to only contain students with 10 or more years of experience at companies with at least 1000 employees, as their recruiter base is suited to more experienced professionals at enterprise companies.

In [35]:
ds_jobs_transformed = ds_jobs_transformed[
    (ds_jobs_transformed['experience']>='10') &
    (ds_jobs_transformed['company_size']>='1000-4999')
]
print(ds_jobs_transformed.shape)
ds_jobs_transformed.head()

(2201, 14)


,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
9,699,city_103,0.919922,Other,True,no_enrollment,Graduate,STEM,17,10000+,Pvt Ltd,>4,123,False
12,25619,city_61,0.913086,Male,True,no_enrollment,Graduate,STEM,>20,1000-4999,Pvt Ltd,3,23,False
31,22293,city_103,0.919922,Male,True,Part time course,Graduate,STEM,19,5000-9999,Pvt Ltd,>4,141,False
34,26494,city_16,0.910156,Male,True,no_enrollment,Graduate,None,12,5000-9999,Pvt Ltd,3,145,False
40,2547,city_114,0.925781,Female,True,Full time course,Masters,STEM,16,1000-4999,Public Sector,2,14,False


In [36]:
ds_jobs_transformed['city'].unique()

array(['city_103', 'city_61', 'city_16', 'city_114', 'city_97', 'city_36',
       'city_157', 'city_89', 'city_28', 'city_165', 'city_136',
       'city_160', 'city_173', 'city_75', 'city_50', 'city_158',
       'city_40', 'city_104', 'city_106', 'city_10', 'city_21', 'city_90',
       'city_102', 'city_67', 'city_37', 'city_91', 'city_71', 'city_145',
       'city_123', 'city_23', 'city_73', 'city_116', 'city_98',
       'city_152', 'city_99', 'city_83', 'city_45', 'city_46', 'city_100',
       'city_65', 'city_76', 'city_144', 'city_150', 'city_159',
       'city_19', 'city_131', 'city_12', 'city_1', 'city_126', 'city_41',
       'city_138', 'city_53', 'city_141', 'city_133', 'city_14',
       'city_167', 'city_27', 'city_57', 'city_115', 'city_149',
       'city_11', 'city_24', 'city_7', 'city_64', 'city_59', 'city_134',
       'city_162', 'city_72', 'city_77', 'city_80', 'city_13', 'city_94',
       'city_175', 'city_107', 'city_105', 'city_118', 'city_69',
       'city_143', 'city